In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Loading the other modules I have in parent directories
import sys
PYTHONPATH="../"
sys.path.append(PYTHONPATH)
from dataProcessing import getDataLoaders
from models import ThreeCNN, ThreeCNN_Module, rnn_2dCNN
from train import check_accuracy, train, train_ThreeCNN
from plottingFcts import trainingMetrics, sigBkgEff, plotROC

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
batch_size = 1024
loader_train, loader_val, loader_test = getDataLoaders(batch_size=batch_size)

In [8]:
layer0_params_2 = [32, 3, 3, 1, 1]
layer0_reduce_2 = [1, 3, 93, 1, 1] # ---> Downsampling with one filter at the end

layer1_params_2 = [32, 3, 3, 1, 1]
layer1_reduce_2 = [1, 12, 9, 1, 1] # ---> Downsampling with one filter 

layer2_params_2 = [32, 3, 3, 1, 1]
layer2_reduce_2 = [1, 12, 3, 1, 1] # ---> downsampling with one filter 

rnn_params_2 = [18, 18, 1]

layer3_params_2 = [32, 3, 3, 1, 1]

fc_params_2 = [18, 150, 100, 50, 3, 0.5]
print(batch_size*rnn_params_2[1])

18432


In [9]:
rnn_model = rnn_2dCNN(layer0_params_2, layer0_reduce_2, layer1_params_2, layer1_reduce_2, layer2_params_2, layer2_reduce_2, rnn_params_2, layer3_params_2, fc_params_2)

optimizer = optim.Adam(rnn_model.parameters(), lr=1e-5)

hist, bestModel = train(loader_train, loader_val, rnn_model, optimizer, epochs=5, returnBest=True)


Epoch 1/5:
Iteration 0, loss = 1.1675
cuda
Got 9932 / 30000 correct (33.11)

Iteration 100, loss = 1.0393
cuda
Got 12509 / 30000 correct (41.70)



KeyboardInterrupt: 